# ECS 289G: Automatic Speech Recongition using OpenAI Whisper model

This notebook conists the code for performing Automatic Speech Recognition using OpenAI whisper-large-v3 model.

In [1]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-sl1og3c6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-sl1og3c6
  Resolved https://github.com/huggingface/transformers.git to commit 2c658b5a4282f2e824b4e23dc3bcda7ef27d5827
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s e

In [2]:
!pip install mltu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 548.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 43.6 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

# Using GPU if available
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Defining the openai/whisper-large-v3 model
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

# Processing the pretrained model
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Testing on validation set of LJ Speech dataset
import typing
import numpy as np
import pandas as pd
from tqdm import tqdm
from mltu.utils.text_utils import ctc_decoder, get_cer, get_wer

# Considering only first 20 data items
df = pd.read_csv("/content/drive/MyDrive/Fall 2023/ECS 289G/Project/Models/05_sound_to_text/20231128/val.csv").values.tolist()
df = df[:20]

accum_cer, accum_wer = [], []

for wav_path, label in tqdm(df):
    # Predicting the output of the audio file supplied in .wav format
    result = pipe(wav_path)
    text = result["text"]

    true_label = "".join([l for l in label.lower() if l in "abcdefghijklmnopqrstuvwxyz'?!. "])

    cer = get_cer(text, true_label)
    wer = get_wer(text, true_label)

    accum_cer.append(cer)
    accum_wer.append(wer)
    print(str(cer) + " " + str(wer))

# Printing Average CER and Avergae WER on LJ Speech dataset
print(f"Average CER: {np.average(accum_cer)}, Average WER: {np.average(accum_wer)}")

  5%|▌         | 1/20 [02:12<41:49, 132.10s/it]

0.13333333333333333 0.5


 10%|█         | 2/20 [04:02<35:45, 119.18s/it]

0.08333333333333333 0.4444444444444444


 15%|█▌        | 3/20 [05:51<32:29, 114.70s/it]

...

0.16455696202531644 0.3125


 95%|█████████▌| 19/20 [35:10<01:49, 109.87s/it]

0.16216216216216217 0.75


100%|██████████| 20/20 [36:55<00:00, 110.77s/it]

0.013333333333333334 0.0
Average CER: 0.08839057300237042, Average WER: 0.27485160104885165


In [4]:
# Testing it with custom audios (the audios which we obatined after running speaker diarization and audio splicing)
import os

directory_path = "/content/drive/MyDrive/Fall 2023/ECS 289G/Project/Dataset/spliced"
output_file_path = "/content/drive/MyDrive/Fall 2023/ECS 289G/Project/all_transcriptions.txt"

files = []

for filename in os.listdir(directory_path):
    files.append(filename)

sorted_files = sorted(files, key=lambda x: int(x.split('_')[-1].split('.')[0]))

# Opening the output file once for writing
with open(output_file_path, 'w') as output_file:
    for filename in sorted_files:
        if os.path.isfile(os.path.join(directory_path, filename)):
            file_path = os.path.join(directory_path, filename)
            result = pipe(file_path)
            speaker_id = filename.split('_')[1]
            transcription_text = result["text"]

            # Writing to the text file
            output_file.write(f"Speaker {speaker_id}: {transcription_text}\n")

            # Printing the transciption
            print(f"Speaker {speaker_id}: {transcription_text}\n")

    print(f"All transcriptions successfully stored in {output_file_path}")

Speaker 00:  you

Speaker 00:  Hey Shreyas, how's your day going?

Speaker 01:  Hi Rohit, my day is going good, how about yours?

Speaker 00:  Oh, that's great. Yeah, even mine's going great. I just got done with my artificial intelligence presentation a while ago, and now I'm working on my project for it. How about you?

Speaker 01:  I am just working on an assignment for my linear algebra course which is due by midnight

Speaker 00:  Oh, great. So, let's see how this audio transcript works in our Hugging Face model.

Speaker 00:  Shud.

All transcriptions successfully stored in /content/drive/MyDrive/Fall 2023/ECS 289G/Project/all_transcriptions.txt
